```python
import tdmclient.notebook
await tdmclient.notebook.list()
await tdmclient.notebook.start()


```python
%%run_python
nf_leds_temperature(0,0)
nf_leds_prox_v(0,0)

In [1]:
from time import sleep
from tdmclient import ClientAsync, aw

def get_status(node):
    status = node.status
    if status == 1:
        return "Connected"
    if status == 2:
        return "Unlocked"
    if status == 3:
        return "Busy"
    if status == 4:
        return "Locked"
    if status == 5:
        return "Disconnected"
    else:
        return "Unknown"

def lock_node(node):
    aw(node.lock())
    sleep(0.1)
    get_status(node)
               
def unlock_node(node):
    aw(node.unlock())
    sleep(0.1)
    get_status(node)

try:
    client = ClientAsync()
    node = aw(client.wait_for_node())
    sleep(0.1)
    
    if node.status != 2:
        print("Please unlock the thymio in Aseba Studio or check connection")
    
    aw(node.wait_for_variables())
    
    try:
        aw(node.lock())
        sleep(0.1)
        aw(node.unlock())
    except:
        print("Please unlock Thymio in Aseba Studio")

    sleep(0.1)
    print(node.props["name"], get_status(node))
except:
    print("Open Aseba Studio and Unlock Thymio / reconnect Thymio")

Super Thymio 474 Unlocked


In [9]:
vars(node)

{'on_variables_changed': {<function tdmclient.clientasynccachenode.ClientAsyncCacheNode.__init__.<locals>.on_variables_changed(node, variables)>},
 'on_events_received': set(),
 'on_event_received': set(),
 'on_vm_state_changed': set(),
 'thymio': <tdmclient.clientasync.ClientAsync at 0x138a855e900>,
 'props': {'node_id': b'\x87\xd4\x8b\xe9+\tIb\xaa\xb4\xb8p\\\x0c\xc9\xe7',
  'node_id_str': '87d48be9-2b09-4962-aab4-b8705c0cc9e7',
  'group_id': b'(B\xb3[g\x11A\x84\xac\xf4YY\x99\xc6[\xc6',
  'group_id_str': '2842b35b-6711-4184-acf4-595999c65bc6',
  'status': 3,
  'type': 1,
  'name': 'Super Thymio 474',
  'capabilities': 3,
  'fw_version': '14'},
 'id': b'\x87\xd4\x8b\xe9+\tIb\xaa\xb4\xb8p\\\x0c\xc9\xe7',
 'id_str': '87d48be9-2b09-4962-aab4-b8705c0cc9e7',
 'status': 3,
 'vm_description': {'node_id': b'\x87\xd4\x8b\xe9+\tIb\xaa\xb4\xb8p\\\x0c\xc9\xe7',
  'node_id_str': '87d48be9-2b09-4962-aab4-b8705c0cc9e7',
  'bytecode_size': 1534,
  'data_size': 640,
  'stack_size': 32,
  'variables': {

In [10]:
node.v.leds.temperature

In [11]:
node.v.leds.top

Node array variable leds.top[3]

In [3]:
PROX_FARLEFT = 0
PROX_LEFT = 1
PROX_FRONT = 2
PROX_RIGHT = 3
PROX_FARRIGHT = 4
PROX_BACKLEFT = 5
PROX_BACKRIGHT = 6

def motors(node, speed_left, speed_right):
    node.v.motor.left.target = speed_left
    node.v.motor.right.target = speed_right

Calibrate robot

In [3]:
lock_node(node)
motors(node, 0,0)
node.flush()
unlock_node(node)

In [4]:
# wifi async
import time

def calibrate_90_left(node, speed):
    time_start = time.time()
    while True:
        motors(node, -speed, speed)
        node.flush() # 
        aw(node.wait_for_variables())
        if node.v.button.center == 1:
            time_elapsed = time.time() - time_start
            motors(node, 0,0)
            print("Exiting...")
            break
        aw(client.sleep(0.01)) # SLEEP TIME
        # low sleep time = good movement but bad response on center button
    return time_elapsed


lock_node(node)
speed = 50
T_L_90 = calibrate_90_left(node, speed)
print(T_L_90)
unlock_node(node)

Exiting...
211.8908417224884


In [13]:
# cable
import time

def calibrate_90_left(node, speed):
    time_0 = time.time()
    time_m = 0
    while True:
        #print(node.v.button.center)
        motors(node, -speed, speed)
        if node.v.button.center == 1:
            time_m = time.time() - time_0
            motors(node, 0,0)
            print("Exiting...")
            break
        node.flush()
        # sleep is not needed
    return time_m

lock_node(node)
aw(client.wait_for_node())
sleep(0.1)
speed = 70
T_L_90 = calibrate_90_left(node, speed)
print(T_L_90)
unlock_node(node)

Exiting...
20.649778604507446


In [ ]:
# cable
import time

def calibrate_90_right(node, speed):
    time_0 = time.time()
    time_m = 0
    while True:
        motors(node, speed, -speed)
        if node.v.button.center == 1:
            time_m = time.time() - time_0
            motors(node, 0,0)
            print("Exiting...")
            break
        node.flush()
    return time_m

lock_node(node)
aw(client.wait_for_node())
sleep(0.1)
speed = 70
T_R_90 = calibrate_90_right(node, speed)
print(T_R_90)
unlock_node(node)

In [ ]:
# cable
import time

def calibrate_semicircle_left(node, speed, speed_factor):
    time_0 = time.time()
    time_m = 0
    while True:
        motors(node, speed_factor*speed, speed)
        if node.v.button.center == 1:
            time_m = time.time() - time_0
            motors(node, 0,0)
            print("Exiting...")
            break
        node.flush()
    return time_m

lock_node(node)
aw(client.wait_for_node())
sleep(0.1)
speed = 70
speed_factor = 2
T_SC_L = calibrate_semicircle_left(node, speed, speed_factor)
print(T_SC_L)
unlock_node(node)

Print Prox to tune prox threshold:

```python
import numpy as np
PROX_FARLEFT = 0
PROX_LEFT = 1
PROX_FRONT = 2
PROX_RIGHT = 3
PROX_FARRIGHT = 4
PROX_BACKLEFT = 5
PROX_BACKRIGHT = 6

lock_node(node)
while True:
    prox_farleft = node.v.prox.horizontal[PROX_FARLEFT]
    prox_left = node.v.prox.horizontal[PROX_LEFT]
    prox_front = node.v.prox.horizontal[PROX_FRONT]
    prox_right = node.v.prox.horizontal[PROX_RIGHT]
    prox_farright = node.v.prox.horizontal[PROX_FARRIGHT]
    prox_backleft = node.v.prox.horizontal[PROX_BACKLEFT]
    prox_backright = node.v.prox.horizontal[PROX_BACKRIGHT]
    
    prox_array = np.array([prox_farleft, prox_left, prox_front, prox_right, prox_farright, prox_backleft, prox_backright])
    print(prox_array)
    node.flush()
    await client.sleep(0.5)

    if node.v.button.center == 1:
        print("Exiting...")
        break

unlock_node(node)
```

A simple prox func

In [ ]:
import numpy as np

def get_prox(node):
    prox_farleft = node.v.prox.horizontal[PROX_FARLEFT]
    prox_left = node.v.prox.horizontal[PROX_LEFT]
    prox_front = node.v.prox.horizontal[PROX_FRONT]
    prox_right = node.v.prox.horizontal[PROX_RIGHT]
    prox_farright = node.v.prox.horizontal[PROX_FARRIGHT]
    prox_backleft = node.v.prox.horizontal[PROX_BACKLEFT]
    prox_backright = node.v.prox.horizontal[PROX_BACKRIGHT]
    
    # just showing if you need numpy, you could do a straightforward mean without numpy
    prox_array = np.array([prox_farleft, prox_left, prox_front, prox_right, prox_farright, prox_backleft, prox_backright]) # (7,)
    #print(np.mean(prox_array))
    if np.mean(prox_array) != 0:
        return True
    else:
        return False

###############################################################################################

lock_node(node)

speed = 100
manual_drive = False

while True:
    if manual_drive:
        node.v.leds.top = [32,32,0]
        node.v.leds.circle = [0,0,0,0,0,0,0,0]
        
        if node.v.button.backward == 1:
            motors(node,-speed,-speed)
            node.v.leds.circle = [0,0,0,0,32,0,0,0]
    
        if node.v.button.left == 1:
            motors(node,-speed,speed)
            node.v.leds.circle = [0,0,0,0,0,0,32,0]
    
    
        if node.v.button.right == 1:
            motors(node,speed,-speed)
            node.v.leds.circle = [0,0,32,0,0,0,0,0]
            
        if node.v.button.forward == 1:
            motors(node,speed,speed)
            node.v.leds.circle = [32,0,0,0,0,0,0,0]
    else:
        node.v.leds.circle = [0,0,0,0,0,0,0,0]
        if get_prox(node):
            motors(node,0,0)
            node.v.leds.top = [32,0,0]
            node.flush()
        else:
            motors(node,speed,speed)
            node.v.leds.top = [0,32,0]

    if node.v.button.center == 1:
        print("Exiting...")
        break
    node.flush()
    #await client.sleep(0.1)

unlock_node(node)

First approach: semi-circle and known size

In [ ]:
def detect_obstacle(node):
    #aw(node.wait_for_variables())
    prox_values = node.v.prox.horizontal

    threshold = 3000

    left_sensors = [prox_values[PROX_FARLEFT], prox_values[PROX_LEFT]]
    left_detected = any(p > threshold for p in left_sensors)

    right_sensors = [prox_values[PROX_FARRIGHT], prox_values[PROX_RIGHT]]
    right_detected = any(p > threshold for p in right_sensors)

    front_sensor = prox_values[PROX_FRONT]
    front_detected = front_sensor > threshold

    if left_detected and not right_detected:
        return 'left'
    elif right_detected and not left_detected:
        return 'right'
    elif front_detected:
        return 'front'
    elif left_detected and right_detected:
        return 'front'
    else:
        return 'none'


while True:
    obstacle = detect_obstacle(node)
    if obstacle == 'left':
        motors(node, speed, -speed)
        node.flush()
        while True:
            obstacle = detect_obstacle(node)
            if obstacle != 'left' and obstacle != 'front':
                break
            sleep(0.1)
        motors(node, speed, speed)
    elif obstacle == 'right':
        node.v.leds.top = [32, 0, 0]
        motors(node, -speed, speed)
        node.flush()
        while True:
            obstacle = detect_obstacle(node)
            if obstacle != 'right' and obstacle != 'front':
                break
            sleep(0.1)
        motors(node, speed, speed)
    elif obstacle == 'front':
        node.v.leds.top = [32, 32, 0]  # Yellow LED
        # Stop or back up slightly
        motors(node, -speed, -speed)
        node.flush()
        sleep(0.5)
        # Try turning right to avoid obstacle
        motors(node, speed, -speed)
        node.flush()
        sleep(0.5)
        motors(node, speed, speed)
    else:
        # No obstacle detected, move forward
        node.v.leds.top = [0, 32, 0]  # Green LED
        motors(node, speed, speed)
if node.v.button.center == 1:
    print("Exiting...")
    break
node.flush()
sleep(0.1)


unlock_node(node)